In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.tools import tool

In [2]:
@tool
def salary_calculator(base_salary:str,bonus_percentage:str)->str:
    """Calculate total salary by adding base salary and bonus amount"""
    try:
        base = float(base_salary.replace("$", "").replace(",", "."))
        bonus = base * float(bonus_percentage.strip("%"))/ 100
        total = base + bonus
        return f"Total Salary: ${total:,.2f} (Base: {base:,.2f} + Bonus:{bonus:,.2f})"
    except Exception as e:
        return f"Error calculating total salary: {e}"
   

In [3]:
@tool
def answer_hr_query(employee_info:str)-> str:
    """Answer general HR questions based on employee information."""
    try:
        return f"Based on employee information, {employee_info}"
    except Exception as e:
        return f"Error processing HR query: {e}"


In [4]:
tools = [salary_calculator, answer_hr_query]

In [5]:
input_documents = []

for filename in os.listdir("./"):
    if filename.endswith(".txt"):
        loader = TextLoader(filename)
        input_documents.extend(loader.load())

In [6]:
input_documents

[Document(page_content='Employee Code: ANL_104\nRole: Business Analyst\nWork Location: Boston\nWork Hours: 9 AM to 5:30 PM EST\nAnnual Leave Policy: 21 days per year\nCompensation: $ 95,000 \nPerformance Bonus: 11% of base salary annually', metadata={'source': 'ANL_104.txt'}),
 Document(page_content='Employee Code: DEV_102\nRole: Software Developer\nWork Location: San Francisco\nWork Hours: 10 AM to 7 PM PST\nAnnual Leave Policy: 20 days per year\nCompensation: $ 100,000 \nPerformance Bonus: 10% of base salary annually', metadata={'source': 'DEV_102.txt'}),
 Document(page_content='Employee Code: DS_105\nRole: Data Scientist\nWork Location: Seattle\nWork Hours: 10 AM to 6:30 PM PST\nAnnual Leave Policy: 20 days per year\nCompensation: $ 110,000 \nPerformance Bonus: 14% of base salary annually', metadata={'source': 'DS_105.txt'}),
 Document(page_content='Employee Code: HR_103\nRole: HR Executive\nWork Location: Chicago\nWork Hours: 9:30 AM to 6:30 PM CST\nAnnual Leave Policy: 22 days per

In [7]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [8]:
vectorstore = FAISS.from_documents(input_documents, embedding)

In [9]:
retriever = vectorstore.as_retriever()

In [10]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

prompt = ChatPromptTemplate.from_messages(
    [ ("system", "You are a HR Assistant. Use the provided employee information to answer questions.\n\n Employee info:{context}"),
     ("human", "{input}"),
     (MessagesPlaceholder(variable_name="agent_scratchpad"))
     ])
   




In [11]:
agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)

In [16]:
# verbose = False => Bize geçici düşünmeyi göstermeyecektir.
agent_executor =AgentExecutor(agent=agent, tools=tools, verbose=True)

In [17]:
employee_code = input("Enter employee code:")
employee_info = retriever.get_relevant_documents(employee_code, k=1)

In [18]:
while True:
    query = input("Enter your query:")
    if query.lower() == "exit":
        print("Exiting the chat...")
        break
    
    result = agent_executor.invoke({"input":query, "context":employee_info})
    print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `salary_calculator` with `{'base_salary': '100000', 'bonus_percentage': '10'}`


Total Salary: $110,000.00 (Base: 100,000.00 + Bonus:10,000.00)The total salary of the Software Developer (DEV_102) role is $110,000. This includes a base salary of $100,000 and a performance bonus of $10,000.

> Finished chain.
The total salary of the Software Developer (DEV_102) role is $110,000. This includes a base salary of $100,000 and a performance bonus of $10,000.
Exiting the chat...
